In [1]:
import os
import pandas as pd 
import numpy as np 
import torch
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer, util

In [2]:
default_path = os.getcwd()
data_path = os.path.join(default_path, '../../data')
model_path = os.path.join(default_path, '../../model')
config_path = os.path.join(default_path, '../../config')

In [3]:
# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [4]:
models = ['sentence-transformers/xlm-r-base-en-ko-nli-ststb', 'sentence-transformers/xlm-r-large-en-ko-nli-ststb', \
          'sentence-transformers/multi-qa-distilbert-dot-v1', 'kakaobank/kf-deberta-base', 'upskyy/kf-deberta-multitask']

In [11]:
prompts = ['금리가 물가에 미치는 영향을 설명해주세요',\
           '금리가 주가에 미치는 영향을 설명해주세요',\
           '미국의 금리 인상이 한국경제에 어떤 영향을 주나요 ?', \
           'PER 주가수익 비율은 무엇인가?',\
           '채권이 무엇인지 설명하고, 채권 투자의 리스크에 대해 알려주세요',\
           '외환위기가 일어날 경우 발생할 일을 과거의 예시를 들어 설명해주세요', \
           '부동산 투자와 주식 투자의 차이점과 리스크에 대해 설명해주세요',\
           '삼성전자와 TSMC의 반도체 시장에서의 경쟁 상황은 어떠한가요 ?', \
           '넷플릭스와 디즈니+ 사이의 스트리밍 경쟁에 대해 어떻게 생각하시나요 ?',\
           '코스피와 코스닥의 차이는 무엇인가요 ?',\
           '블랙-숄즈 모델(Black-Scholes Model)이 무엇인지 설명해주세요',\
           '금융 위기 동안 사용된 양적 완화(Quantitive Easing)의 원리와 그 장단점은 무엇인가요?',\
           '고빈도 거래(High-Frequency Trading, HFT)의 원리와 그것이 주식시장에 미치는 영향에 대해 설명해주세요', \
           '신용 스프레드(Credit Spread) 변동의 주요 원인은 무엇이며, 이것이 채권 투자자에게 어떤 영향을 가지나요 ?',\
           '한국은행의 기능과 역할에 대해 설명해보시오']

In [33]:
toks = [] 

for idx, p in enumerate(prompts): 
    toks.append([])
    for model_name in models:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        tokens = tokenizer.tokenize(p)
        # encoded_token = tokenizer.encode(p)
        # decoded_token = tokenizer.decode(encoded_token)
        toks[idx].append(tokens)

In [34]:
emb_result = pd.DataFrame(toks, index=prompts, columns=models)
emb_result.head(5)

,sentence-transformers/xlm-r-base-en-ko-nli-ststb,sentence-transformers/xlm-r-large-en-ko-nli-ststb,sentence-transformers/multi-qa-distilbert-dot-v1,kakaobank/kf-deberta-base,upskyy/kf-deberta-multitask
금리가 물가에 미치는 영향을 설명해주세요,"[▁금, 리가, ▁물, 가, 에, ▁미, 치는, ▁영향을, ▁설명, 해주세요]","[▁금, 리가, ▁물, 가, 에, ▁미, 치는, ▁영향을, ▁설명, 해주세요]","[ᄀ, ##ᅳ, ##ᆷ, ##ᄅ, ##ᅵ, ##ᄀ, ##ᅡ, ᄆ, ##ᅮ, ##ᆯ,...","[금리, ##가, 물가, ##에, 미치, ##는, 영향, ##을, 설명, ##해, ...","[금리, ##가, 물가, ##에, 미치, ##는, 영향, ##을, 설명, ##해, ..."
금리가 주가에 미치는 영향을 설명해주세요,"[▁금, 리가, ▁주, 가, 에, ▁미, 치는, ▁영향을, ▁설명, 해주세요]","[▁금, 리가, ▁주, 가, 에, ▁미, 치는, ▁영향을, ▁설명, 해주세요]","[ᄀ, ##ᅳ, ##ᆷ, ##ᄅ, ##ᅵ, ##ᄀ, ##ᅡ, ᄌ, ##ᅮ, ##ᄀ,...","[금리, ##가, 주가, ##에, 미치, ##는, 영향, ##을, 설명, ##해, ...","[금리, ##가, 주가, ##에, 미치, ##는, 영향, ##을, 설명, ##해, ..."
미국의 금리 인상이 한국경제에 어떤 영향을 주나요 ?,"[▁미국의, ▁, 금리, ▁인상, 이, ▁한국, 경제, 에, ▁어떤, ▁영향을, ▁...","[▁미국의, ▁, 금리, ▁인상, 이, ▁한국, 경제, 에, ▁어떤, ▁영향을, ▁...","[ᄆ, ##ᅵ, ##ᄀ, ##ᅮ, ##ᆨ, ##ᄋ, ##ᅴ, ᄀ, ##ᅳ, ##ᆷ,...","[미국, ##의, 금리, 인상, ##이, 한국, ##경제, ##에, 어떤, 영향, ...","[미국, ##의, 금리, 인상, ##이, 한국, ##경제, ##에, 어떤, 영향, ..."
PER 주가수익 비율은 무엇인가?,"[▁PER, ▁주, 가, 수, 익, ▁비율, 은, ▁무엇인가, ?]","[▁PER, ▁주, 가, 수, 익, ▁비율, 은, ▁무엇인가, ?]","[per, ᄌ, ##ᅮ, ##ᄀ, ##ᅡ, ##ᄉ, ##ᅮ, ##ᄋ, ##ᅵ, ##...","[PER, 주가, ##수익, 비율, ##은, 무엇, ##인가, ?]","[PER, 주가, ##수익, 비율, ##은, 무엇, ##인가, ?]"
"채권이 무엇인지 설명하고, 채권 투자의 리스크에 대해 알려주세요","[▁채, 권, 이, ▁무, 엇, 인지, ▁설명, 하고, ,, ▁채, 권, ▁투자, ...","[▁채, 권, 이, ▁무, 엇, 인지, ▁설명, 하고, ,, ▁채, 권, ▁투자, ...","[ᄎ, ##ᅢ, ##ᄀ, ##ᅯ, ##ᆫ, ##ᄋ, ##ᅵ, [UNK], ᄉ, ##...","[채권, ##이, 무엇, ##인지, 설명, ##하고, ,, 채권, 투자, ##의, ...","[채권, ##이, 무엇, ##인지, 설명, ##하고, ,, 채권, 투자, ##의, ..."


In [38]:
default_path = os.getcwd()
data_path = os.path.join(default_path, '../../../data')

In [39]:
emb_result.to_csv(os.path.join(data_path, 'emb_result.csv'))

### MTEB

In [1]:
from mteb import MTEB
from sentence_transformers import SentenceTransformer

# Define the sentence-transformers model name
model_name = "average_word_embeddings_komninos"

model = SentenceTransformer(model_name)
evaluation = MTEB(tasks=["FiQA2018"])
results = evaluation.run(model, output_folder=f"results/{model_name}")

.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/267M [00:00<?, ?B/s]

(…)beddings/whitespacetokenizer_config.json:   0%|          | 0.00/2.59M [00:00<?, ?B/s]

(…)WordEmbeddings/wordembedding_config.json:   0%|          | 0.00/164 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

...

───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Retrieval

- FiQA2018, beir, s2p

Error while evaluating FiQA2018: Retrieval tasks require beir package. Please install it with `pip install mteb[beir]`


Exception: Retrieval tasks require beir package. Please install it with `pip install mteb[beir]`